In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
154,ABW,North America,Aruba,2020-08-19,1121.0,0.0,57.714,4.0,0.0,0.143,...,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29
387,AFG,Asia,Afghanistan,2020-08-19,37599.0,0.0,47.143,1375.0,0.0,4.429,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
538,AGO,Africa,Angola,2020-08-19,1966.0,60.0,41.000,90.0,2.0,1.714,...,1.362,5819.495,NaN,276.045,3.94,NaN,NaN,26.664,NaN,61.15


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-18,France,221493.0,0.0,30453.0,0.0,0.0,0.0
0,2020-08-18,China,89511.0,0.0,4709.0,0.0,0.0,0.0
0,2020-08-18,Italy,254778.0,0.0,35426.0,0.0,0.0,0.0
0,2020-08-18,Spain,360772.0,0.0,28658.0,0.0,0.0,0.0
0,2020-08-18,United States,5534405.0,0.0,171401.0,0.0,0.0,0.0
0,2020-08-18,World,22456722.0,0.0,782646.0,0.0,0.0,0.0
0,2020-08-18,United Kingdom,320399.0,0.0,41430.0,0.0,0.0,0.0
0,2020-08-18,Germany,224643.0,0.0,9240.0,0.0,0.0,0.0
0,2020-08-18,Iran,349430.0,0.0,20047.0,0.0,0.0,0.0
0,2020-08-18,Turkey,251590.0,0.0,6013.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-19,France,221493.0,221267.0,30453.0,30451.0,226.0,2.0
0,2020-08-19,China,89511.0,89494.0,4709.0,4705.0,17.0,4.0
0,2020-08-19,Italy,254778.0,254636.0,35426.0,35405.0,142.0,21.0
0,2020-08-18,Spain,360772.0,364196.0,28658.0,28670.0,-3424.0,-12.0
0,2020-08-19,United States,5534405.0,5482416.0,171401.0,171821.0,51989.0,-420.0
0,2020-08-19,World,22456722.0,22151281.0,782646.0,781123.0,305441.0,1523.0
0,2020-08-19,United Kingdom,320399.0,320286.0,41430.0,41381.0,113.0,49.0
0,2020-08-19,Germany,224643.0,226914.0,9240.0,9243.0,-2271.0,-3.0
0,2020-08-19,Iran,349430.0,347835.0,20047.0,19972.0,1595.0,75.0
0,2020-08-19,Turkey,251590.0,251805.0,6013.0,6016.0,-215.0,-3.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")